# Конфигурация графика

Altair предоставляется различные способы изменения внешнего вида графика. Мы не будем рассматривать все возможные вариант, но рассмотрим какие настройки, в принципе, существуют.

В общем случае, внешний вид графика определяется следующими составляющими (в порядке возрастания приоритета).

1. **Настройка верхнего уровня**. На верхнем уровне настройки графиков Altair, можно определить настройки конфигурации, применимые к каждой панели и слою графика.

2. **Настройки локального уровня**. Настройки верхнего уровня могут быть локально переопределены

3. **Значения кодирования**. Если значения кодирования заданы, они имеют высший приоритет и переопределяют остальные настройки.

Рассмотрим пример.

In [1]:
import altair as alt

In [2]:
import numpy as np
import pandas as pd
np.random.seed(42)

data = pd.DataFrame(np.random.randn(100, 2), columns=['x', 'y'])

## Пример 1: Настройка свойств маркеров

Предположим, что мы хотим определить цвет маркеров в диаграмме рассеяния. Рассмотрим все три варианта для этого. Используем набор данных с нормально распределёнными точками:

In [3]:
alt.Chart(data).mark_point().encode(
    x='x:Q',
    y='y:Q'
)

alt.Chart(...)

### Настройки верхнего уровня

На верхнем уровне у Altair есть метод ``configure_mark()``, который позволяет настроить множество различных параметров меток, а также свойство ``configure_point()`` которое определяет свойства именно точек.

Ниже приведены возможные варианты настроек из документации:

In [4]:
alt.Chart.configure_point?

Эти настройки высокого уровня могут быть интерпретированы как тема графика, они используются как настройки по умолчанию для визуальных отображений всех элементов графика.
Используем ``configure_point`` для задания некоторых свойств точек:

In [5]:
alt.Chart(data).mark_point().encode(
    x='x:Q',
    y='y:Q'
).configure_point(
    size=200,
    color='red',
    filled=True
)

alt.Chart(...)

## Локальные настройки маркеров

В фукнции ``mark_point()`` можно передать локальные настройки, которые перепишут настройки верхнего уровня.
Аргументы у этой функции те же, что и для ``configure_mark``.

In [6]:
alt.Chart(data).mark_point(color='green', filled=False).encode(
    x='x:Q',
    y='y:Q'
).configure_point(
    size=200,
    color='red',
    filled=True
)

alt.Chart(...)

Обратите внимание, что настройки "color" и "fill" переписаны локальными, а "size" остался прежним.

## Настройки кодирования

И, наконец, высочайший приоритет настроек у кодирования. Зададим цвет равный "steelblue":

In [7]:
alt.Chart(data).mark_point(color='green', filled=False).encode(
    x='x:Q',
    y='y:Q',
    color=alt.value('steelblue')
).configure_point(
    size=200,
    color='red',
    filled=True
)

alt.Chart(...)

## Пример 2: Подписи к осям и заголовки

Подписи к осям и заголовок графика устанавливаются автоматически на основе исходных данных, но иногда мы хотим поменять их.
Например, визуализируем гистограмму данных из примера выше:

In [8]:
alt.Chart(data).mark_bar().encode(
    x=alt.X('x', bin=True),
    y=alt.Y('count()')
)

alt.Chart(...)

Можно вручную задать подписи осей с использованием аргумента ``title``:

In [9]:
alt.Chart(data).mark_bar().encode(
    x=alt.X('x', bin=True, title='binned x values'),
    y=alt.Y('count()', title='counts in x')
)

alt.Chart(...)

Аналогично, можно задать свойство ``title`` для всего графика:

In [10]:
alt.Chart(data).mark_bar().encode(
    x=alt.X('x', bin=True, title='binned x values'),
    y=alt.Y('count()', title='counts in x')
).properties(
    title='A histogram'
)

alt.Chart(...)

## Пример 3: Свойства осей

Для задания свойств осей (в том числе линий сетки), можно использовать аргумент ``axis``.

In [11]:
alt.Chart(data).mark_bar().encode(
    x=alt.X('x', bin=True, axis=alt.Axis(labelAngle=45)),
    y=alt.Y('count()', axis=alt.Axis(labels=False, ticks=False, title=None))
)

alt.Chart(...)

Некоторые из этих значений могут быть заданы и на верхнем уровне настрок, если есть необходимость применить их к графику в целом, например:

In [12]:
alt.Chart(data).mark_bar().encode(
    x=alt.X('x', bin=True),
    y=alt.Y('count()', axis=alt.Axis(labels=False, ticks=False, title=None))
).configure_axisX(
    labelAngle=45
)

alt.Chart(...)

## Пример 4: Параметры шкалы и пределы осей

Каждое кодирование также содержит параметр ``scale``, который позволяет задать такие параметры как пределы осей и другие свойства шкалы.

In [13]:
alt.Chart(data).mark_point().encode(
    x=alt.X('x:Q', scale=alt.Scale(domain=[-5, 5])),
    y=alt.Y('y:Q', scale=alt.Scale(domain=[-5, 5])),
)

alt.Chart(...)

Заметим, что если уменьшить масштаб до размеров, меньших чем реальные пределы значений, по умолчанию данные выйдут за пределы шкалы:

In [14]:
alt.Chart(data).mark_point().encode(
    x=alt.X('x:Q', scale=alt.Scale(domain=[-3, 1])),
    y=alt.Y('y:Q', scale=alt.Scale(domain=[-3, 1])),
)

alt.Chart(...)

Иногда полезно не скрывать такие данные, чтобы не упустить случайно точки данных.

Если вы всё же хотите ограничить визуализируемые данные пределами шкалы, можно задать параметр ``clip``:

In [15]:
alt.Chart(data).mark_point(clip=True).encode(
    x=alt.X('x:Q', scale=alt.Scale(domain=[-3, 1])),
    y=alt.Y('y:Q', scale=alt.Scale(domain=[-3, 1])),
)

alt.Chart(...)

Вместо этого можно использовать параметр "clamp", чтобы ограничить данные шкалой, сохраняя их видимыми даже за пределами шкалы:

In [16]:
alt.Chart(data).mark_point().encode(
    x=alt.X('x:Q', scale=alt.Scale(domain=[-3, 1], clamp=True)),
    y=alt.Y('y:Q', scale=alt.Scale(domain=[-3, 1], clamp=True)),
).interactive()

alt.Chart(...)

## Пример 5: Цветовые шкалы

Иногда полезно вручную задать цветовую шкалу

In [17]:
from vega_datasets import data
weather = data.seattle_weather()
weather.head()

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


In [18]:
alt.Chart(weather).mark_point().encode(
    x='date:T',
    y='temp_max:Q',
    color='weather:N'
)

alt.Chart(...)

Можно изменить цветовую схему, используя свойство шкалы, либо используя одну из [именованных цветовых схем Vega](https://vega.github.io/vega/docs/schemes/#reference):

In [19]:
alt.Chart(weather).mark_point().encode(
    x='date:T',
    y='temp_max:Q',
    color=alt.Color('weather:N', scale=alt.Scale(scheme="dark2"))
)

alt.Chart(...)

Либо можно создать свою цветовую шкалу, задав цветовой домен и пределы:

In [20]:
colorscale = alt.Scale(domain=['sun', 'fog', 'drizzle', 'rain', 'snow'],
                       range=['goldenrod', 'gray', 'lightblue', 'steelblue', 'midnightblue'])

alt.Chart(weather).mark_point().encode(
    x='date:T',
    y='temp_max:Q',
    color=alt.Color('weather:N', scale=colorscale)
)

alt.Chart(...)

## Упражнение: Настройка графиков

Возьмите понравившиеся графики из предыдущего блокнота и настройте для них:

- метки,
- заголовок графика и подписи осей,
- пределы осей x и y.

In [38]:
alt.Chart(weather).mark_point().encode(
    x=alt.X('date:T', title='Дата'),
    y=alt.Y('temp_max:Q', scale=alt.Scale(domain=[5, 20], clamp=True),
           title='Максимальная температура'),
    color='weather:N'
).properties(
    title='График максимальной температуры по времени'
).interactive()

alt.Chart(...)

In [37]:
data = pd.DataFrame(np.random.randn(100, 2), columns=['x', 'y'])
alt.Chart(data).mark_point(color='black', filled=True).encode(
    x=alt.X('x:Q', scale=alt.Scale(domain=[-1, 1],clamp=True),
           title='Рандомное X'),
    y=alt.Y('y:Q', scale=alt.Scale(domain=[0, 1], clamp=True),
           title='Рандомное Y')
).configure_point(
    size=200,
    color='red',
    filled=True
).properties(
    title='Рандомные числа'
).interactive()

alt.Chart(...)